# `sourmash tax` subcommands

**for integrating taxonomic information**

The sourmash `tax` or `taxonomy` commands integrate taxonomic information into the results of `sourmash gather`. `tax` commands  require a properly formatted `taxonomy` csv file that corresponds to the database used for `gather`. For supported databases (e.g. GTDB, NCBI), we provide these files, but they can also be generated for user-generated databases. For more information, see [databases](databases.md).

These commands rely upon the fact that `gather` results are non-overlapping: the fraction match for gather on each query will be between 0 (no database matches) and 1 (100% of query matched). We use this property to aggregate gather matches at the desired taxonomic rank. For example, if the gather results for a metagenome include results for 30 different strains of a given species, we can sum the fraction match to each strain to obtain the fraction match to this species.

As with all reference-based analysis, results can be affected by the completeness of the reference database. However, summarizing taxonomic results from `gather` minimizes issues associated with increasing size and redundancy of reference databases.

### All `tax` commands require a lineage file for the sourmash database used with `gather`

#### download and look at gtdb-rs202 lineage file

In [1]:
!curl -L https://osf.io/p6z3w/download -o gtdb-rs202.taxonomy.v2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   483  100   483    0     0    133      0  0:00:03  0:00:03 --:--:--   133
100 35.2M  100 35.2M    0     0  3018k      0  0:00:11  0:00:11 --:--:-- 11.3M


In [2]:
!head gtdb-rs202.taxonomy.v2.csv

ident,superkingdom,phylum,class,order,family,genus,species
GCF_014075335.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_002310555.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_900013275.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_000168095.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_002459845.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_001614695.1,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Escherichia,s__Escherichia flexneri
GCF_000356585.2,d__Bacteria,p__Proteobact

### All `tax` commands work on the output files from `sourmash gather`

example gather output:

In [3]:
!head outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv

intersect_bp,f_orig_query,f_match,f_unique_to_query,f_unique_weighted,average_abund,median_abund,std_abund,name,filename,md5,f_match_orig,unique_intersect_bp,gather_result_rank,remaining_bp,query_filename,query_name,query_md5,query_bp
442000,0.08815317112086159,0.08438335242458954,0.08815317112086159,0.05815279361459521,1.6153846153846154,1.0,1.1059438185997785,"GCF_001881345.1 Escherichia coli strain=SF-596, ASM188134v1",/group/ctbrowngrp/gtdb/databases/ctb/gtdb-rs202.genomic.k31.sbt.zip,683df1ec13872b4b98d59e98b355b52c,0.042779713511420826,442000,0,4572000,outputs/abundtrim/HSMA33MX.abundtrim.fq.gz,HSMA33MX,9687eeed,5014000
390000,0.07778220981252493,0.10416666666666667,0.07778220981252493,0.050496823586903404,1.5897435897435896,1.0,0.8804995294906566,"GCF_009494285.1 Prevotella copri strain=iAK1218, ASM949428v1",/group/ctbrowngrp/gtdb/databases/ctb/gtdb-rs202.genomic.k31.sbt.zip,1266c86141e3a5603da61f57dd863ed0,0.052236806857755155,390000,1,4182000,outputs/abundtrim/HSMA33MX.abundtr

## `sourmash tax summarize` (for summarizing metagenomes)

 **for each gather query, summarize gather results by taxonomic lineage.**

There are three possible output formats, `summary`, `lineage_summary`, and `krona`.

- `summary` is the default output format. This outputs a `csv` with lineage summarization for each taxonomic rank. This output currently consists of four columns, `query_name,rank,fraction,lineage`, where `fraction` is the  fraction of the query matched to the reported rank and lineage.
- `krona` format is a tab-separated list of these results at a specific rank. The first column, `fraction` is the fraction of the query matched to the reported rank and lineage. The remaining columns are `superkingdom`, `phylum`, .. etc down to the rank used for summarization. This output can be used directly for summary visualization.
- `lineage_summary` - the lineage summary format is most useful when comparing across metagenomes. each row is a lineage at the desired reporting rank. The columns are each query used for gather, with the fraction match reported for each lineage. This format is commonly used as input for many external multi-sample visualization tools.

example `lineage_summary`:

    lineage    sample1  sample2 sample3
    lin_a     0.4    0.17     0.6
    lin_b     0.0    0.0      0.1
    lin_c     0.3    0.4      0.2

In [4]:
!sourmash tax summarize -h


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

usage:  summarize [-h] [--from-file FILE] [-q] [-o OUTPUT_BASE] -t FILE
                  [FILE ...] [--keep-full-identifiers]
                  [--keep-identifier-versions] [--fail-on-missing-taxonomy]
                  [--output-format {summary,krona,lineage_summary} [{summary,krona,lineage_summary} ...]]
                  [-r {species,genus,family,order,class,phylum,superkingdom}]
                  [-f]
                  [gather_results ...]

positional arguments:
  gather_results

optional arguments:
  -h, --help            show this help message and exit
  --from-file FILE      input many gather results as a text file, with one
                        gather csv per line
  -q, --quiet           suppress non-error output
  -o OUTPUT_BASE, --output-base OUTPUT_BASE
                        base filepath for output file(s) (default stdout)
  -t FILE [FILE ...], --t

## Use `summarize` to aggregate taxonomic information for gather results

In [5]:
!sourmash tax summarize outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv -t gtdb-rs202.taxonomy.v2.csv


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 2 lineage assignments.
The following are missing from the taxonomy information: GCA_002754635,GCA_000256725
query_name,rank,fraction,lineage
HSMA33MX,superkingdom,0.131,d__Bacteria
HSMA33MX,phylum,0.073,d__Bacteria;p__Bacteroidota
HSMA33MX,phylum,0.058,d__Bacteria;p__Proteobacteria
HSMA33MX,class,0.073,d__Bacteria;p__Bacteroidota;c__Bacteroidia
HSMA33MX,class,0.058,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria
HSMA33MX,order,0.073,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales
HSMA33MX,order,0.058,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales
HSMA33MX,family,0.073,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae
HSMA33MX,family,0.058,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae
HSMA33MX,genus,0.058,d__Bac

the default `summarize` output current reports the summarization at all ranks

Note: `of 6, missed 2 lineage assignments.`

This gather used additional databases -- we need to add additional taxonomies to properly summarize!

In [6]:
!curl -L https://osf.io/cbhgd/download -o bacteria_genbank_lineages.csv
!curl -L https://osf.io/q8j6d/download -o bacteria_refseq_lineages.csv
!curl -L https://osf.io/urtfx/download -o protozoa_genbank_lineages.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   459  100   459    0     0    802      0 --:--:-- --:--:-- --:--:--     0 --:--:-- --:--:--   803
100 83.1M  100 83.1M    0     0  13.0M      0  0:00:06  0:00:06 --:--:-- 18.0M  0:00:05  0:00:02 12.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   459  100   459    0     0   1242      0 --:--:-- --:--:-- --:--:--  1240
100 24.2M  100 24.2M    0     0  2858k      0  0:00:08  0:00:08 --:--:-- 5074k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   459  100   459    0     0   1249      0 --:--:-- --:--:-- --:--:--  1247
100  107k  100  107k    0     0  45242      0  0:00:02  0:00:02 --:--:--  125k


In [7]:
%%bash
head protozoa_genbank_lineages.csv
head bacteria_refseq_lineages.csv

accession,taxid,superkingdom,phylum,class,order,family,genus,species,strain
GCA_004431415,2762,Eukaryota,,Glaucocystophyceae,,Cyanophoraceae,Cyanophora,Cyanophora paradoxa,
GCA_000150955,556484,Eukaryota,Bacillariophyta,Bacillariophyceae,Naviculales,Phaeodactylaceae,Phaeodactylum,Phaeodactylum tricornutum,Phaeodactylum tricornutum CCAP 1055/1
GCA_000310025,2880,Eukaryota,,Phaeophyceae,Ectocarpales,Ectocarpaceae,Ectocarpus,Ectocarpus siliculosus,
GCA_000194455,2898,Eukaryota,,Cryptophyceae,Cryptomonadales,Cryptomonadaceae,Cryptomonas,Cryptomonas paramecium,
GCA_000372725,280463,Eukaryota,Haptista,Haptophyta,Isochrysidales,Noelaerhabdaceae,Emiliania,Emiliania huxleyi,Emiliania huxleyi CCMP1516
GCA_001939145,2951,Eukaryota,,Dinophyceae,Suessiales,Symbiodiniaceae,Symbiodinium,Symbiodinium microadriaticum,
GCA_900617105,2996,Eukaryota,,Chrysophyceae,Hydrurales,Hydruraceae,Hydrurus,Hydrurus foetidus,
GCA_001638955,158060,Eukaryota,Euglenozoa,Euglenida,Euglenales,Euglenaceae,Euglena,Euglena g

... and now run `summarize` with multiple taxonomies.

In [8]:
!sourmash tax summarize outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv --taxonomy-csv bacteria_refseq_lineages.csv protozoa_genbank_lineages.csv gtdb-rs202.taxonomy.v2.csv


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 0 lineage assignments.
query_name,rank,fraction,lineage
HSMA33MX,superkingdom,0.245,Eukaryota
HSMA33MX,superkingdom,0.131,d__Bacteria
HSMA33MX,phylum,0.245,Eukaryota;Apicomplexa
HSMA33MX,phylum,0.073,d__Bacteria;p__Bacteroidota
HSMA33MX,phylum,0.058,d__Bacteria;p__Proteobacteria
HSMA33MX,class,0.222,Eukaryota;Apicomplexa;Aconoidasida
HSMA33MX,class,0.073,d__Bacteria;p__Bacteroidota;c__Bacteroidia
HSMA33MX,class,0.058,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria
HSMA33MX,class,0.023,Eukaryota;Apicomplexa;Conoidasida
HSMA33MX,order,0.222,Eukaryota;Apicomplexa;Aconoidasida;Haemosporida
HSMA33MX,order,0.073,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales
HSMA33MX,order,0.058,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales
HSMA33MX,order,0.023,Eukaryota;Apicomplexa;Conoidasida;Eucoccidiori

## Produce Krona-compatible summary at species-level

In [9]:
!mkdir -p outputs/tax
!sourmash tax summarize -r species outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv -t gtdb-rs202.taxonomy.v2.csv --output-format krona -o outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 2 lineage assignments.
The following are missing from the taxonomy information: GCA_002754635,GCA_000256725


In [10]:
!ktImportText outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.krona.tsv

Writing text.krona.html...


### `sourmash tax classify` (for classifying genomes)

`sourmash tax classify` - for each gather query, report likely classification based on `gather` matches. By default, classification requires at least 10% of the query to be matched. Thus, if 10% of the query was matched to a species, the species-level classification can be reported. However, if 7% of the query was matched to one species, and an additional 5% matched to a different species in the same genus, the genus-level classification will be reported.

Optionally, `classify` can instead report classifications at a desired `rank`, regardless of match threshold. 

Note that these thresholds and strategies are under active testing.

There are two possible output formats, `summary` and `krona`.

- `summary` is the default output format. This outputs a `csv` with lineage summarization for each taxonomic rank. This output currently consists of four columns, `query_name,rank,fraction,lineage`, where `fraction` is the  fraction of the query matched to the reported rank and lineage.
- `krona` format is a tab-separated list of these results at a specific rank. The first column, `fraction` is the fraction of the query matched to the reported rank and lineage. The remaining columns are `superkingdom`, `phylum`, .. etc down to the rank used for summarization. This output can be used directly for summary visualization.

In [11]:
%%bash
sourmash tax classify -h

usage:  classify [-h] [-q] -t FILE [FILE ...] [--from-file FILE]
                 [-o OUTPUT_BASE]
                 [-r {species,genus,family,order,class,phylum,superkingdom}]
                 [--keep-full-identifiers] [--keep-identifier-versions]
                 [--fail-on-missing-taxonomy]
                 [--output-format {summary,krona} [{summary,krona} ...]] [-f]
                 [--containment-threshold CONTAINMENT_THRESHOLD]
                 [gather_results ...]

positional arguments:
  gather_results

optional arguments:
  -h, --help            show this help message and exit
  -q, --quiet           suppress non-error output
  -t FILE [FILE ...], --taxonomy-csv FILE [FILE ...]
                        database lineages csv
  --from-file FILE      input many gather results as a text file, with one
                        gather csv per line
  -o OUTPUT_BASE, --output-base OUTPUT_BASE
                        base filepath for output file(s) (default stdout)
  -r {species,genus,fa


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==



In [12]:
!sourmash tax classify outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv -t gtdb-rs202.taxonomy.v2.csv


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 2 lineage assignments.
The following are missing from the taxonomy information: GCA_000256725,GCA_002754635
loaded 0 gather files for classification.
query_name,status,rank,fraction,lineage
HSMA33MX,match,superkingdom,0.131,d__Bacteria


In [13]:
!sourmash tax classify --rank species outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv -t gtdb-rs202.taxonomy.v2.csv --output-format krona -o outputs/tax.HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.classify


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 2 lineage assignments.
The following are missing from the taxonomy information: GCA_002754635,GCA_000256725
loaded 0 gather files for classification.


### `sourmash tax label` (for labeling gather results)

`sourmash tax label` - for any gather results, add a column with taxonomic lineage information for each database match. Do not summarize or classify. Note that this is not required for either `summarize` or `classify`.

By default, `label` uses the name of each input gather csv to write an updated version with lineages information. For example, labeling `sample1.gather.csv` would produce `sample1.gather.with-lineages.csv`

In [14]:
%%bash
sourmash tax label -h

usage:  label [-h] [-q] [--from-file FILE] -t FILE [FILE ...] [-o OUTPUT_DIR]
              [--keep-full-identifiers] [--keep-identifier-versions]
              [--fail-on-missing-taxonomy] [-f]
              [gather_results ...]

positional arguments:
  gather_results

optional arguments:
  -h, --help            show this help message and exit
  -q, --quiet           suppress non-error output
  --from-file FILE      input many gather results as a text file, with one
                        gather csv per line
  -t FILE [FILE ...], --taxonomy-csv FILE [FILE ...]
                        database lineages csv
  -o OUTPUT_DIR, --output-dir OUTPUT_DIR
                        directory for output csv(s)
  --keep-full-identifiers
                        do not split identifiers on whitespace
  --keep-identifier-versions
                        after splitting identifiers, do not remove accession
                        versions
  --fail-on-missing-taxonomy
                        fail quickl


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==



In [15]:
!sourmash tax label outputs/gather/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.csv -t gtdb-rs202.taxonomy.v2.csv --output-dir outputs/tax


== This is sourmash version 4.1.1.dev84+g7044ae5. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==

loaded 6 gather results.
of 6, missed 2 lineage assignments.
The following are missing from the taxonomy information: GCA_002754635,GCA_000256725


In [16]:
%%bash
ls outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31*

outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.krona.tsv
outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.krona.txt
outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.with-lineages.csv


In [17]:
!head outputs/tax/HSMA33MX_gather_x_gtdbrs202_genbank_euks_k31.with-lineages.csv

intersect_bp,f_orig_query,f_match,f_unique_to_query,f_unique_weighted,average_abund,median_abund,std_abund,name,filename,md5,f_match_orig,unique_intersect_bp,gather_result_rank,remaining_bp,query_filename,query_name,query_md5,query_bp,lineage
442000,0.08815317112086159,0.08438335242458954,0.08815317112086159,0.05815279361459521,1.6153846153846154,1.0,1.1059438185997785,"GCF_001881345.1 Escherichia coli strain=SF-596, ASM188134v1",/group/ctbrowngrp/gtdb/databases/ctb/gtdb-rs202.genomic.k31.sbt.zip,683df1ec13872b4b98d59e98b355b52c,0.042779713511420826,442000,0,4572000,outputs/abundtrim/HSMA33MX.abundtrim.fq.gz,HSMA33MX,9687eeed,5014000,d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacterales;f__Enterobacteriaceae;g__Escherichia;s__Escherichia coli
390000,0.07778220981252493,0.10416666666666667,0.07778220981252493,0.050496823586903404,1.5897435897435896,1.0,0.8804995294906566,"GCF_009494285.1 Prevotella copri strain=iAK1218, ASM949428v1",/group/ctbrowngrp/gtdb/databases/ct